<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1">Introduction</a></span></li><li><span><a href="#Loading-the-data" data-toc-modified-id="Loading-the-data-2">Loading the data</a></span><ul class="toc-item"><li><span><a href="#Load-all-the-data-and-randomly-split-it-into-1500-training-examples-and-500-validation-examples." data-toc-modified-id="Load-all-the-data-and-randomly-split-it-into-1500-training-examples-and-500-validation-examples.-2.1">Load all the data and randomly split it into 1500 training examples and 500 validation examples.</a></span></li></ul></li><li><span><a href="#Sparse-Representations" data-toc-modified-id="Sparse-Representations-3">Sparse Representations</a></span><ul class="toc-item"><li><span><a href="#Function-that-converts-an-example-(e.g.-a-list-of-words)-into-a-sparse-bag-of-words-representation:" data-toc-modified-id="Function-that-converts-an-example-(e.g.-a-list-of-words)-into-a-sparse-bag-of-words-representation:-3.1">Function that converts an example (e.g. a list of words) into a sparse bag-of-words representation:</a></span></li></ul></li><li><span><a href="#Support-Vector-Machine-via-Pegasos" data-toc-modified-id="Support-Vector-Machine-via-Pegasos-4">Support Vector Machine via Pegasos</a></span><ul class="toc-item"><li><span><a href="#Pegasos-algorithm" data-toc-modified-id="Pegasos-algorithm-4.1">Pegasos algorithm</a></span></li><li><span><a href="#Support-Vector-Machine-(SVM)" data-toc-modified-id="Support-Vector-Machine-(SVM)-4.2">Support Vector Machine (SVM)</a></span></li><li><span><a href="#Implementation-of-the-Pegasos-algorithm-to-run-on-a-sparse-data-representation" data-toc-modified-id="Implementation-of-the-Pegasos-algorithm-to-run-on-a-sparse-data-representation-4.3">Implementation of the Pegasos algorithm to run on a sparse data representation</a></span><ul class="toc-item"><li><span><a href="#Version-1" data-toc-modified-id="Version-1-4.3.1">Version 1</a></span></li><li><span><a href="#Version-2" data-toc-modified-id="Version-2-4.3.2">Version 2</a></span></li></ul></li><li><span><a href="#Comparison-of-the-two-approaches" data-toc-modified-id="Comparison-of-the-two-approaches-4.4">Comparison of the two approaches</a></span></li><li><span><a href="#Loss-function" data-toc-modified-id="Loss-function-4.5">Loss function</a></span></li></ul></li><li><span><a href="#Regularization-parameter-optimization" data-toc-modified-id="Regularization-parameter-optimization-5">Regularization parameter optimization</a></span></li></ul></div>

# Homework 3: SVM and Sentiment Analysis

Assignment for the [Foundations of Machine Learning](https://bloomberg.github.io/foml/#about) course.
The assignment text can be found in the SVM folder under hw3.pdf

## Introduction

- Goal of the assignment: perform sentiment analysis on movie reviews. We will be working with **natural language data**. 
- First, the **Pegasos algorithm** is implemented. Pegasos is essentially stochastic subgradient descent for the SVM with a particular schedule for the step-size.
- Second, because in natural language domains we typically have huge feature spaces, we work with **sparse representations** of feature vectors, where only the non-zero entries are explicitly recorded. This will require coding the gradient and SGD code using hash tables (dictionaries in Python), rather than numpy arrays.
- Then we will perform feature engineering to improve our model.

## Loading the data

- We will be using the [Polarity Dataset v2.0](https://www.cs.cornell.edu/people/pabo/movie-review-data/), constructed by Pang and Lee. 
- It has the full text from 2000 movies reviews: 1000 reviews are classified as “positive” and 1000 as “negative”. Our goal is to predict whether a review has positive or negative sentiment from the text of the review. 
- Each review is stored in a separate file: the positive reviews are in a folder called “pos”, and the negative reviews are in “neg”. 

### Load all the data and randomly split it into 1500 training examples and 500 validation examples.

In [1]:
################################Code provided in the assignment###################################################
import os
import numpy as np
import pickle
import random

'''
Note:  This code is just a hint for people who are not familiar with text processing in python. 
There is no obligation to use this code, though you may if you like. 
'''


def folder_list(path,label):
    '''
    PARAMETER PATH IS THE PATH OF YOUR LOCAL FOLDER
    '''
    filelist = os.listdir(path)
    review = []
    for infile in filelist:
        file = os.path.join(path,infile)
        r = read_data(file)
        r.append(label)
        review.append(r)
    return review

def read_data(file):
    '''
    Read each file into a list of strings. 
    Example:
    ["it's", 'a', 'curious', 'thing', "i've", 'found', 'that', 'when', 'willis', 'is', 'not', 'called', 'on', 
    ...'to', 'carry', 'the', 'whole', 'movie', "he's", 'much', 'better', 'and', 'so', 'is', 'the', 'movie']
    '''
    f = open(file)
    lines = f.read().split(' ') #split each line on whitespace
    symbols = '${}()[].,:;+-*/&|<>=~" '
    words = map(lambda Element: Element.translate(str.maketrans("", "", symbols)).strip(), lines)
    words = filter(None, words)
    list_words = list(words)
    return list_words

###############################################
######## YOUR CODE STARTS FROM HERE. ##########
###############################################

def shuffle_data():
    '''
    pos_path is where you save positive review data.
    neg_path is where you save negative review data.
    '''
    pos_path = '/Users/Coline/Desktop/mlprojects/Bloomberg_foundations_of_ML/hw3-svm/data/pos'
    neg_path = '/Users/Coline/Desktop/mlprojects/Bloomberg_foundations_of_ML/hw3-svm/data/neg'
    
    pos_review = folder_list(pos_path,1)
    neg_review = folder_list(neg_path,-1)

    review = pos_review + neg_review
    random.shuffle(review)
  
    '''
    Now you have read all the files into list 'review' and it has been shuffled.
    Save your shuffled result by pickle.
    *Pickle is a useful module to serialize a python object structure. 
    *Check it out. https://wiki.python.org/moin/UsingPickle
    '''
    train = review[:1500]
    valid = review[1500:]
    
    with open('train.pkl', 'wb') as f:
        pickle.dump(train, f)
    
    with open('valid.pkl', 'wb') as f:
        pickle.dump(valid, f)
    
    return train, valid

In [2]:
train, valid = shuffle_data()

In [3]:
#load files from memory
with open('train.pkl', 'rb') as f:
    train = pickle.load(f)
with open('valid.pkl', 'rb') as f:
    valid = pickle.load(f)

## Sparse Representations

- The most basic way to represent text documents for machine learning is with a **“bag-of-words”** representation. 
- Here every possible word is a feature, and the value of a word feature is the number of times that word appears in the document. Of course, most words will not appear in any particular document, and those counts will be zero. 
- Rather than store a huge number of zeros, we use a **sparse representation**, in which we only store the counts that are nonzero. The counts are stored in a key/value store (such as a dictionary in Python).  
For example, “Harry Potter and Harry Potter II” would be represented as the following Python dict: ``x={’Harry’:2, ’Potter’:2, ’and’:1, ’II’:1}``.  
- We will be using linear classifiers of the form $f(x) = w^T x$, and we can store the $w$ vector in a sparse format as well, such as ``w={’minimal’:1.3, ’Harry’:-1.1, ’viable’:-4.2, ’and’:2.2, ’product’:9.1}``. 
- The inner product between $w$ and $x$ would only involve the features that appear in both $x$ and $w$, since whatever doesn’t appear is assumed to be zero. For this example, the inner product would be ``x[Harry] * w[Harry] + x[and] * w[and] = 2*(-1.1) + 1*(2.2)``.  

### Function that converts an example (e.g. a list of words) into a sparse bag-of-words representation:

In [4]:
from collections import Counter

def bag_of_words(review):
    '''
    Converts a list of words into a sparse bag-of-words representation.
    @Param review: list (iterable)
    Returns a dictionary with keys=words and values=number of times the words appear
    '''
    cnt = Counter()
    for word in review:
        cnt[word] += 1
    return cnt

## Support Vector Machine via Pegasos

In this question you will build an SVM using the Pegasos algorithm. To align with the notation used in the [Pegasos paper](https://ttic.uchicago.edu/~nati/Publications/PegasosMPB.pdf), we’re considering the following formulation of the SVM objective function: $$ min_{w \in R^d} \frac{\lambda}{2} + \frac{1}{m} \sum_{i=1}^m max\{0, 1-y_iw^Tx_i\} $$

Note that, for simplicity, we are leaving off the unregularized bias term $b$. 

### Pegasos algorithm
Pegasos is stochastic subgradient descent using a step size rule $\eta_t = \frac{1}{\lambda t}$. The pseudocode is given below:

***
Input: $\lambda > 0$. Choose $w_1 = 0; t = 0$

While termination condition not met  

&nbsp;&nbsp;For $j = 1,...,m$ (assumes data is randomly permuted)  

&nbsp;&nbsp;&nbsp;&nbsp;$t = t + 1$  

&nbsp;&nbsp;&nbsp;&nbsp;$\eta_t = \frac{1}{t\lambda}$;  

&nbsp;&nbsp;&nbsp;&nbsp;If $y_j w^T_t x_j < 1$  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$w_{t+1} = (1 - \eta_t \lambda)w_t + \eta_t y_j x_j$  

&nbsp;&nbsp;&nbsp;&nbsp;Else  
&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;$w_{t+1} = (1 - \eta_t \lambda)w_t$ 
***

### Support Vector Machine (SVM)
The “stochastic” SVM objective function is the SVM objective function with a single training point:  
  
$$J_i(w) = \frac{\lambda}{2} \|w\|^2 + max\{0, 1-y_iw^Tx_i\}$$   
The function  $J_i(w)$ is not differentiable at $y_i w^T x_i = 1$ .

A subgradient of $J_i(w)$ is given by 

$$ g =
  \begin{cases}
    \lambda w - y_i x_i       & \quad \text{for } y_i w^T x_i < 1 \\
    \lambda w  & \quad \text{for } y_i w^T x_i \geq 1
  \end{cases}
$$

If the step size rule is $\eta_t = \frac{1}{\lambda t}$, then doing SGD with the subgradient direction from the previous problem is the same as given in the pseudocode.

### Implementation of the Pegasos algorithm to run on a sparse data representation

In [5]:
#########################################code given in the assignment#############################################
# Taken from http://web.stanford.edu/class/cs221/ Assignment #2 Support Code

def dotProduct(d1, d2):
    """
    @param dict d1: a feature vector represented by a mapping from a feature (string) to a weight (float).
    @param dict d2: same as d1
    @return float: the dot product between d1 and d2
    """
    if len(d1) < len(d2):
        return dotProduct(d2, d1)
    else:
        return sum(d1.get(f, 0) * v for f, v in d2.items())

def increment(d1, scale, d2):
    """
    Implements d1 += scale * d2 for sparse vectors.
    @param dict d1: the feature vector which is mutated.
    @param float scale
    @param dict d2: a feature vector.

    NOTE: This function does not return anything, but rather
    increments d1 in place. We do this because it is much faster to
    change elements of d1 in place than to build a new dictionary and
    return it.
    """
    for f, v in d2.items():
        d1[f] = d1.get(f, 0) + v * scale
        
#########################################end of code given in the assignment#####################################

#### Version 1

In [6]:
def pegasos(train, epoch, l_reg=0.1):
    '''
    Implements Pegasos algorithm running on sparse data representation. 
    @param train: list of lists, each list contains single words from the review
    @param epoch: number of epochs, integer
    @param l_reg: regularization parameter lambda, float
    
    The weight dictionnary w is initialized as an empty dictionnary. 
    For each epoch, the algorithm passes through each datapoint of the randomized dataset and the step size 
    is decreased by a factor of 1/t (t increments after each datapoint).
    Each datapoint is transformed into a bag of words using the bag_of_words function, and the label y_j is 
    extracted. The gradient is calculated using the appropriate subgradient, defined by the value of the
    margin (calculated using the dotProduct function). The weight dictionnary is updated with the gradient
    and step size using the increment function.
    '''
    w = {}
    t = 0

    for i in range(epoch):
        
        if i%10 == 0:
            print("Epoch: {}".format(i))
        
        for j in range(len(train)):
            t = t+1
            step = 1/(l_reg*t)
            x_j = bag_of_words(train[j][:-1])
            y_j = train[j][-1]                   #The label is the last entry in the list
            margin = y_j*dotProduct(w,x_j)
        
            if margin < 1:
                increment(w, -step*l_reg, w)
                increment(w, step*y_j, x_j)
            
            else:
                increment(w, -step*l_reg, w)
        
    return w

In [7]:
w1 = pegasos(train, epoch=2, l_reg=0.01)

Epoch: 0


#### Version 2

- Note that in every step of the Pegasos algorithm, we rescale every entry of $w_t$ by the factor $(1 − \eta_t \lambda)$. 
- Implementing this directly with dictionaries is very slow. 
- We can make things significantly faster by representing $w$ as $w = sW$, where $s \in R$ and $W \in R^d$. 
- You can start with $s = 1$ and $W$ all zeros (i.e. an empty dictionary). Note that both updates (i.e. whether or not we have a margin error) start with rescaling $w_t$, which we can do simply by setting $s_{t+1} = (1 − \eta_t \lambda)s_t$. If the update is $w_{t+1} = (1 − \eta_t \lambda)w_t + \eta_t y_j x_j$, then **verify that the Pegasos update step is equivalent to**:  

$$s_{t+1} = (1 − \eta_t \lambda)s_t$$
$$W_{t+1} = W_t + \frac{1}{s_{t+1}} \eta_t y_j x_j.$$

There is one subtle issue with the approach described above: if we ever have $1 − \eta_t \lambda = 0$, then $s_{t+1} = 0$, and we’ll have a divide by 0 in the calculation for $W_{t+1}$. This only happens when $\eta_t = 1/\lambda$. With our step-size rule of $\eta_t = 1/\lambda t$, it happens exactly when $t = 1$. So one approach is to just start at $t = 2$. More generically, note that if $s_{t+1} = 0$, then $w_{t+1} = 0$. Thus an equivalent representation is $s_{t+1} = 1$ and $W = 0$. Thus if we ever get $s_{t+1} = 0$, simply set it back to 1 and reset $W_{t+1}$ to zero, which is an empty dictionary in a sparse representation.  
Implement the Pegasos algorithm with the $(s, W)$ representation described above. [See section 5.1 of Leon Bottou’s Stochastic Gradient Tricks for a more generic version of this technique, and many other useful tricks.]

In [8]:
def pegasos_v2(train, valid, epoch, l_reg=0.1, tol=1e-9):
    '''
    pegasos_v2 function implements Pegasos algorithm running on sparse data representation. 
    @param train: list of lists, each list contains single words from the review. Training set
    @param valid: list of lists, each list contains single words from the review. Validation set
    @param epoch: number of epochs, integer
    @param l_reg: regularization parameter lambda, float
    @param tol: tolerance level for estimation of convergence, float
    
    The weight dictionnary w is represented as w=sW, where s in is R and W is in Rd. 
    The parameters initialization is s=1 and W empty dictionary.
    '''
    s = 1
    W = {}
    t = 1
    w = {}

    for i in range(epoch):
         
        #Storing the weight vector to perform estimation of convergence after each epoch
        w_old = W.copy() 
        for a,b in w_old.items():
            w_old[a] = b*s_new

        for j in range(len(train)):
            t = t+1
            step = 1/(l_reg*t)
            x_j = bag_of_words(train[j][:-1])
            y_j = train[j][-1]
             
            margin = s*y_j*dotProduct(W,x_j)
            
            if margin < 1:
                s_new = s*(1-step*l_reg)
                increment(W, (step*y_j)/s_new, x_j)
                s = s_new
            
            else:
                s_new = s*(1-step*l_reg)
                s = s_new    
            
        
        if i%100 == 0:
            print("Epoch: {}".format(i))
            w = W.copy()
            for a,b in w.items():
                w[a] = b*s_new
            loss_val = loss(valid,w)
            print('The model classified {}% of the examples correctly'.format(loss_val))
        
        #Testing for convergence:
        if i != 0: 
            list_weights_diff = []
            w = W.copy()
            for a,b in w.items():
                w[a] = b*s_new
            for k, v in w_old.items():
                list_weights_diff.append(np.abs(w.get(k, 0) - w_old[k])) 
        
            if np.all(np.asarray(list_weights_diff) < tol):
                print('converged')
                break   
    return w

### Comparison of the two approaches

In [9]:
%time w1 = pegasos(train, epoch=2, l_reg=0.01)

Epoch: 0
CPU times: user 23.1 s, sys: 184 ms, total: 23.3 s
Wall time: 23.8 s


In [10]:
%time w2 = pegasos_v2(train, valid, epoch=2, l_reg=0.01)

Epoch: 0


NameError: name 'loss' is not defined

In [11]:
loss(valid, w1)

NameError: name 'loss' is not defined

In [ ]:
loss(valid, w2)

In [ ]:
#Looking at the content of the weight dicts:
list_weights_diff = []
for k, v in w1.items():
    list_weights_diff.append(np.abs(w2.get(k, 0) - w1[k]))
np.all(np.asarray(list_weights_diff) < 1e-2)

- When expressing the vector $w_t$ as the product $s_tW_t$, where s is a scalar, as was done in pegasos_v2 algorithm, the algorithm is sped up by a factor of 14, which is significantly faster.
- The performance of the algorithms was equivalent: they had the same classification error on the training set, and the weight vectors were very similar.
- In the rest of this assignement, I will use pegasos_v2

### Loss function

In [12]:
def loss(valid, w):
    '''
    function that takes a sparse weight vector w and a collection of (x,y) pairs,
    and returns the percent error when predicting y using sign(w^Tx). 
    In other words, the function reports the 0-1 loss of the linear predictor x -> w^Tx.
    '''
    count = 0
    for i in range(len(valid)):
        y_i = valid[i][-1]
        x_i = bag_of_words(valid[i][:-1])
        pred = dotProduct(w,x_i)
        
        if (pred/y_i) > 0: #occurs when y_i and w^Tx_i have the same sign (ie correct prediction)
            count += 1
            
    percent = (count/(len(valid)))*100
    
    return percent

## Regularization parameter optimization

- Using the bag-of-words feature representation described above, search for the regularization parameter that gives the minimal percent error on your test set. 
- Using the faster Pegasos implementation, running to convergence. 

In [ ]:
#Range of lambda values
l_reg_arr = np.unique(np.concatenate((10.**np.arange(-6,1,1), np.arange(1,3,.3))))

#Initialize values
w_history = []
loss_history = []

#Run SVM algo for each lambda
for l_reg in l_reg_arr:
    w = pegasos_v2(train, valid, epoch=250, l_reg=l_reg, tol=1e-2)
    w_history.append(w)
    
    #Calculate misclassification percentage for each lambda
    loss_val = 100-loss(valid, w)
    loss_history.append(loss_val)

In [ ]:
#Present the results in a table
import pandas as pd
df_error = pd.DataFrame({'Param_l_reg': l_reg_arr, 'Classification error': loss_history})

print(df_error)

min_idx = df_error['Classification error'].idxmin()
print('The regression parameter that minimizes test loss was {:.2e}.'.format(df_error['Param_l_reg'][min_idx]))

In [ ]:
import matplotlib.pyplot as plt
# Plot validation performance vs regularization parameter
fig, ax = plt.subplots()

ax.semilogx(df_error["Param_l_reg"], df_error["Classification error"])
ax.grid()
ax.set_title("Test Performance vs Regularization")
ax.set_xlabel("L-Penalty Regularization Parameter")
ax.set_ylabel("Classification error")

In summary, the regularization parameters in the range [1e-4,1e-2] achieved the best performance for the model.